In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [ ]:
URL = 'https://ryman.com/events/'

response = requests.get(URL)

In [ ]:
type(response)

In [ ]:
response.status_code

In [ ]:
response.text

1. Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all performers? For now, just worry about the headliner and don't worry about the opener. (Eg. For Vince Gill, featuring Wendy Moten, we only care about Vince Gill.) Make use of this to create a list containing just the names of each inductee.

In [ ]:
ryman_soup = BS(response.text)

In [ ]:
#we just look for a tags where class=tribe-event-url, which if you look, all of the a tags with band titles have this class
a_tags_2 = ryman_soup.findAll('a', attrs = {'class':'tribe-event-url'})
a_tags_2

In [ ]:
#try to extract the title of just one before putting in a list comp
a_tag_1 = a_tags_2[0]
a_tag_1.get('title')

In [ ]:
titles_2 = [x.get('title') for x in a_tags_2]
titles_2

2. Next, try and find a tag that could be used to find the date and time for each show. Extract these into two lists, one containing the date and the other containing the time. (Eg. THURSDAY, AUGUST 4, 2022 AT 8:00 PM CDT should be split into August 4, 2022 and 8:00 PM CDT.) 

In [ ]:
time = ryman_soup.findAll('time')
#type(time)
time

In [ ]:
#extract text only from ResultSet
time_text = [x.text for x in time]
time_text

extract the SHOWTIME

In [ ]:
#try slicing on one before loop, it works
time_1 = time_text[0]
time_1[-11:]

In [ ]:
showtime_texts = [x[-11:] for x in time_text]
showtime_texts

In [ ]:
#figure out how to slice one date
date_1 = time_text[0]
print(date_1)
date_1[:date_1.index(' at')]

In [ ]:
date_texts = [x[:x.index(' at')] for x in time_text]
date_texts

3. Take the two lists you created on parts 1 and 2 and convert it into a pandas DataFrame.

In [ ]:
ryman_df_pg1 = pd.DataFrame({'band':titles_2, 'showtime':showtime_texts, 'date':date_texts})
ryman_df_pg1

4. Now, you need to take what you created for the first page and apply it across multiple rest of the pages so that you can scrape all inductees. Notice how the url changes when you click the "More Events" button at the top of the page. Check that the code that you wrote for the first page still works for page 2. Once you have verified that your code will still work, write a for loop that will cycle through the first five pages of events.

In [ ]:
URL = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged=2'
req = requests.get(URL)
#req.status_code
ryman_soup_2 = BS(req.text)

In [ ]:
a_pg2 = ryman_soup_2.findAll('a', attrs = {'class':'tribe-event-url'})
titles_pg2 = [x.get('title') for x in a_pg2]
titles_pg2


In [ ]:
URL2 = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='
#so you can see what's happening in the loop: it iterates through the 5 web pages we want
for page in range (1,6):
    print(URL2 + str(page))
    print(page)

In [ ]:
URL2 = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='

artist_list = []

for page in range (1,6): 
    req = requests.get(URL2 + str(page))
    soup = BS(req.text)
    a_tags = soup.findAll('a', attrs = {'class':'tribe-event-url'})
    titles = [x.get('title') for x in a_tags]
    artist_list.extend(titles)    

In [ ]:
artist_list

In [ ]:
URL2 = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='

showtime_list = []

for page in range (1,6):
    req = requests.get(URL2 + str(page))
    soup = BS(req.text)
    time = soup.findAll('time')
    time_text = [x.text for x in time]
    showtimes = [x[-11:] for x in time_text]
    showtime_list.extend(showtimes)

In [ ]:
showtime_list

In [ ]:
URL2 = 'https://ryman.com/events/list/?tribe_event_display=list&tribe_paged='

date_list = []

for page in range (1,6):
    req = requests.get(URL2 + str(page))
    soup = BS(req.text)
    time = soup.findAll('time')
    time_text = [x.text for x in time]
    dates = [x[:x.index(' at')] for x in time_text]
    date_list.extend(dates)

In [ ]:
date_list

In [ ]:
#end result is a dataframe of the band, showtime and date for events from the first 5 web pages
ryman_df = pd.DataFrame({'band':artist_list, 'showtime':showtime_list, 'date':date_list})
ryman_df